# Constrained Orthogonal Matching Pursuit for Audio Declipping

In [11]:
import numpy as np
import matplotlib.pyplot as plt

## Signal Clipping

In [33]:
def clip(y, theta_clip):
    return np.clip(y, -theta_clip, theta_clip)

In [34]:
def get_theta_clip(y):
    return np.max(np.abs(y))

## Gabor Dictionary

In [30]:
params = {
    "sampling_rate": 16000,
    "frame_length": 1024,
    "frame_overlap": 768
}

# Number of atoms
K_g = params["frame_length"]
# Length of the signal
N = params["frame_length"]

In [31]:
# Create the time-frequency grid
T = np.arange(0, N)
J = np.arange(0, K_g)
J, T = np.meshgrid(J, T)

# Dictionaries of shape (N, K_g)
gabor_cosine = np.cos(np.pi * (J+1/2) * (T+1/2) / K_g)
gabor_sine = np.sin(np.pi * (J+1/2) * (T+1/2) / K_g)

## Orthogonal Matching Pursuit

In [32]:
def least_squares(y, D_c, D_s, theta_clip = None, theta_max = None):
    """
    Solves the following least squares problem
        min ||y - D_c * x_c - D_s * x_s||^2
    """
    x = np.linalg.lstsq(D, y, rcond=None)[0]
    return x

In [ ]:
def OMP(y, M_r, D_c, D_s, K, eps, theta_clip = None, theta_max = None):
    """ 
    Runs the Orthogonal Matching Pursuit algorithm, using Gabor Dictionaries

    Inputs:
    --------
    y: np.array
        Input signal of size N_r
    M_r: np.array
        Measurement matrix of size (N_r, N)
    D_c: np.array
        Dictionary for the cosine atoms of size (N, K_g)
    D_s: np.array
        Dictionary for the sine atoms of size (N, K_g)
    K: int
        Maximal number of atoms to select
    eps: float
        Stopping criterion
    theta_clip: float
        (Optional) Clipping value of the signal, used as an additional constraint in the least squares problem. If None, no clipping constraint is applied.
    theta_max: float
        (Optional) Maximum value of the signal, used as an additional constraint in the least squares problem. If None, no maximum constraint is applied.

    Outputs:
    --------
    x: np.array
        Estimated sparse signal
    """

    # Intialization
    N = D_c.shape[0]
    K_g = D_c.shape[1]
    N_r = M_r.shape[0]

    ## Dictionaries
    W_c = np.linalg.inv(np.linalg.norm(M_r @ D_c, axis=0))
    W_s = np.linalg.inv(np.linalg.norm(M_r @ D_s, axis=0))
    d_c_norm = M_r @ D_c @ W_c
    d_s_norm = M_r @ D_s @ W_s
    d_cs_dot = np.diag(np.dot(d_c_norm.T, d_s_norm), k = 0)           # Array containing <d_j^c|d_j^s>, j = 0, ..., K_g-1

    ## Residual and support
    r = y
    Omega = []

    for k in range(K):

        # Atom selection (TODO)
        x_c = (np.dot(d_c_norm.T, r) - d_cs_dot * np.dot(d_s_norm.T, r)) / (1 - d_cs_dot**2)
        x_s = (np.dot(d_s_norm.T, r) - d_cs_dot * np.dot(d_c_norm.T, r)) / (1 - d_cs_dot**2)
        proj = np.zeros(d_c_norm.shape[0])
        for j in range(d_c_norm.shape[0]):
            proj[j] = np.linalg.norm(r - x_c[j] * d_c_norm[j,:] - x_s[j] * d_s_norm[j,:])**2
        i = np.argmax(np.abs(proj))

        # Update support and residual (TODO)
        Omega.append(i)
        x_hat = least_squares(y, d_c_norm[Omega,:], d_s_norm[Omega,:], theta_clip, theta_max)
        r = y - np.dot(d_c_norm[Omega,:], x_hat) - np.dot(d_s_norm[Omega,:], x_hat)

        # Stopping criterion
        if np.linalg.norm(r) < eps:
            break
    
    # Output (TODO)
    x = np.zeros(N)
    return x

## Data

### Dataset

### Exploratory Data Analysis

## Experiments